In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import cv2
import os
from tqdm import tqdm
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from detector.input_pipeline import DetectorPipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

dataset_path = '/home/dan/datasets/COCO/multiposenet/train/'
filenames = os.listdir(dataset_path)
filenames = [n for n in filenames if n.endswith('.tfrecords')]
filenames = [os.path.join(dataset_path, n) for n in sorted(filenames)]

params = {
    'batch_size': 16, 'image_height': 640, 'image_width': 640, 'min_dimension': 640
}
with tf.device('/cpu:0'):
    pipeline = DetectorPipeline(filenames, is_training=True, params=params)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()
    features.update(labels)

# Show an image

In [ ]:
def draw_boxes(image, boxes):
    
    image = Image.fromarray(image).copy()
    draw = ImageDraw.Draw(image, 'RGBA')

    for box in boxes:
        ymin, xmin, ymax, xmax = box
        fill = (255, 0, 0, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )

    return image

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

In [ ]:
i = 0
image = (255.0*output['images'][i]).astype('uint8')
boxes = output['boxes'][i].copy()

num_boxes = output['num_boxes'][i]
boxes = boxes[:num_boxes]

h, w, _ = image.shape
scaler = np.array([h, w, h, w], dtype='float32')
boxes *= scaler

draw_boxes(image, boxes)

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run(features)
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())

# Measure box scale distribution

In [ ]:
# when running this don't forget to set `image_size` very small (like 128)

datasets_size = pipeline.num_examples
num_batches_per_epoch = datasets_size // 16
num_epochs = 10
num_steps = num_epochs * num_batches_per_epoch

result = []
with tf.Session() as sess:
    sess.run(init)        
    for _ in tqdm(range(num_steps)):
        output = sess.run(features)
        boxes = output['boxes']
        num_boxes = output['num_boxes']
        result.append((boxes, num_boxes))

In [ ]:
scales = []
for b, n in result:
    ymin, xmin, ymax, xmax = np.split(b, 4, axis=2)
    h, w = ymax - ymin, xmax - xmin
    s = np.sqrt(h * w)
    s = np.squeeze(s, axis=2)
    for i in range(len(n)):
        scales.append(s[i][:n[i]])
        
scales = np.concatenate(scales)

In [ ]:
plt.hist(scales, bins=100);